In [1]:
# Initilizing the SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('KMeans').getOrCreate()

In [2]:
# Importing the dataset
df = spark.read.csv('/FileStore/tables/hack_data.csv', inferSchema=True, header=True)
df = df.drop('Location')

In [3]:
# Transform the dataset
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=df.columns, outputCol='features')
df = assembler.transform(df)

In [4]:
# Feature scaling
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
df = scaler.fit(df).transform(df)

In [5]:
# Creating the model
from pyspark.ml.clustering import KMeans
kmeans = KMeans(featuresCol='scaled_features', k=2)
kmeans = kmeans.fit(df)

In [6]:
# Getting the predictions
kmeans.transform(df).show(3)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed| features| scaled_features|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
 8.0| 391.09| 1| 2.96| 7.0| 72.37|[8.0,391.09,1.0,2...|[0.56785108466505...| 1|
 20.0| 720.99| 0| 3.04| 9.0| 69.08|[20.0,720.99,0.0,...|[1.41962771166263...| 1|
 31.0| 356.32| 1| 3.71| 8.0| 70.58|[31.0,356.32,1.0,...|[2.20042295307707...| 1|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
only showing top 3 rows

In [7]:
# Evaluating the model (k = 2)
wssse = kmeans.computeCost(df)
centers = kmeans.clusterCenters()
print(wssse)
print(centers)

601.7707512676716
[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
 3.28474 ]), array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
 5.26676612])]

In [8]:
# Counting the number of samples in each predicted cluster (k = 2)
df_pred = kmeans.transform(df)
df_pred = df_pred.groupby('prediction').count()
df_pred.show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [9]:
# Evaluating the model (k = 3)
# wssse = kmeans.computeCost(df)
# centers = kmeans.clusterCenters()
# print(wssse)
# print(centers)

434.1492898715845
[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
 3.28474 ]), array([1.21780112, 1.37901802, 1.99757683, 1.37198977, 2.55237797,
 5.29152222]), array([1.30217042, 1.25830099, 0. , 1.35793211, 2.57251009,
 5.24230473])]

In [10]:
# Counting the number of samples in each predicted cluster (k = 3)
# df_pred = kmeans.transform(df)
# df_pred = df_pred.groupby('prediction').count()
# df_pred.show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+